Script to plot matrice of min NEPs over a range of durations-areas

In [1]:
import sys
import os
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs

from matplotlib.lines import Line2D
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

from config import *
from utils import proj
from functions import get_data_yr_SAFR
from make_acc_nep_SAFR import load_nep_SAFR_sub2

R[write to console]: Le chargement a nécessité le package : kde1d



In [2]:
### CST ###

reg = 'Fr'
dataset = 'SAFRAN'
ymin = 1958
ymax = 2021
years = np.arange(ymin, ymax+1, 1)
per = str(ymin) + '-' + str(ymax)
nbweeksmin = 4
nbweeksmax = 166  # 52  # 260
ts = str(nbweeksmin) + '-' + str(nbweeksmax)
mm = np.arange(1, 12+1,1)
typ = 'kernall'  # Method to compute NEPs
sizes = {10: 5, 20: 2, 30: 1, 40: 1} #, 50: 1, 60: 1}  # domain side length (pixels) and NEP mat sub-sampling factor
prop = 0.33  # proportion du côté de la fenêtre sans recouvrement (dans les deux directions)
th_n = 0.75  # minimum proportion of land grid cell for the sub-domain to be considered
projmap = ccrs.LambertConformal(central_longitude=2.337229, central_latitude=46.8, false_easting=600000., false_northing=2200000.)

In [3]:
# OUTDIR

if not os.path.isdir(DATADIR + '/' + reg):
    os.mkdir(DATADIR + '/' + reg)
datadir = DATADIR + '/' + reg

if not os.path.isdir(datadir + '/' + dataset):
    os.mkdir(datadir + '/' + dataset)
datadir = datadir + '/' + dataset

if not os.path.isdir(datadir + '/' + per):
    os.mkdir(datadir + '/' + per)
datadir = datadir + '/' + per


if not os.path.isdir(FIGDIR + '/' + reg):
    os.mkdir(FIGDIR + '/' + reg)
figdir = FIGDIR + '/' + reg

if not os.path.isdir(figdir + '/' + dataset):
    os.mkdir(figdir + '/' + dataset)
figdir = figdir + '/' + dataset

if not os.path.isdir(figdir + '/' + per):
    os.mkdir(figdir + '/' + per)
figdir = figdir + '/' + per

In [7]:
figdir

'/home/guillaumechagnaud/TRAVAIL/POSTDOC/IGE-Secheresse/work/output/figures/Fr/SAFRAN/1958-2021'

In [5]:
if not os.path.isdir(datadir + '/nep_multi-domain_meth2'):
    os.mkdir(datadir + '/nep_multi-domain_meth2')
datadir_nep = datadir + '/nep_multi-domain_meth2'

# Get minimum NEPs and location for each size
nep_min_sizes = {}
loc_min_sizes = {}

for size, fac in sizes.items():
    size_ = str(int(size*res/1000)) + 'x' + str(int(size*res/1000))   # sub-domain size (km x km)

    print(size_, end=' : ', flush=True)

    data = load_nep_SAFR_sub2(reg, ymin, ymax, typ, nbweeksmin, nbweeksmax, size=size)
    nep_min_sizes[size] = data[0]  # matrix dates x timescales with nep values
    loc_min_sizes[size] = data[1]  # matrix dates x timescales with indices of sub-domain with min


    #~ Save

    datafile_nep_min = datadir_nep + '/' + ts + '_' + size_

    #with open(datafile_nep_min, 'wb') as pics:
    #    pickle.dump(obj=nep_min_size, file=pics)

80x80 : 160x160 : 240x240 : 320x320 : 

In [8]:
data

(                 4         5         6         7         8         9    \
 1958-08-01       NaN       NaN       NaN       NaN       NaN       NaN   
 1958-08-02       NaN       NaN       NaN       NaN       NaN       NaN   
 1958-08-03       NaN       NaN       NaN       NaN       NaN       NaN   
 1958-08-04       NaN       NaN       NaN       NaN       NaN       NaN   
 1958-08-05       NaN       NaN       NaN       NaN       NaN       NaN   
 ...              ...       ...       ...       ...       ...       ...   
 2022-07-27  0.108353  0.375745  0.358852  0.234413  0.231698  0.148860   
 2022-07-28  0.084967  0.273743  0.353834  0.227156  0.221803  0.148544   
 2022-07-29  0.082342  0.155325  0.379907  0.247338  0.233657  0.161302   
 2022-07-30  0.091260  0.127662  0.395833  0.260743  0.226961  0.170196   
 2022-07-31  0.089833  0.084307  0.394385  0.260678  0.202017  0.169309   
 
                  10        11        12        13   ...       157       158  \
 1958-08-01       

In [6]:
# Plot matrix of NEP (duration x area)

cmap = plt.cm.get_cmap('viridis_r')

ncols = [0., 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.]
norm = BoundaryNorm(ncols, ncolors=cmap.N, clip=True)

xtlbs = np.arange(nbweeksmin, nbweeksmax+4, 4)
ytlbs = [int(round(size*res/1000, 0)) for size in sizes]
xtcks = np.arange(0, nbweeksmax-nbweeksmin+1, 4)
ytcks = np.arange(0.5, len(sizes), 1)


if not os.path.isdir(figdir + '/nep_multi-domain_meth2'):
    os.mkdir(figdir + '/nep_multi-domain_meth2')
figdir_nep = figdir + '/nep_multi-domain_meth2'


for y in years:
    print(y, end=' : ', flush=True)

    fig, ax = plt.subplots(figsize=(7.5, 5.5))
    
    plt.subplots_adjust(bottom=0.15, top=0.9)
    
    nep_mins = []
    for size in sizes:
        # SELECT SAFRAN HYDROLOGICAL YEARS: 1/8/Y - 31/7/Y+1 !!!!!!!
        nep_min_size = nep_min_sizes[size][nep_min_sizes[size].index.year == y].min(axis=0)
        nep_mins.append(nep_min_size)
        
        #sys.exit()

    nep_mins = np.array(nep_mins)
    
    ax.pcolor(nep_mins, norm=norm, cmap=cmap)

    ax.set_xticks(xtcks)
    ax.set_yticks(ytcks)
    ax.set_xticklabels(xtlbs, rotation=60)
    ax.set_yticklabels(ytlbs, rotation=60)
    ax.set_xlabel('Accumulation period [weeks]')
    ax.set_ylabel('Area side length [km]')
    ax.set_title(y)
    
    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), orientation='horizontal', label='NEP', pad=0.25)
    cb.set_ticks(ncols)
    cb.set_ticklabels(ncols)
    
    figfile = figdir_nep + '/' + str(y) + '.pdf'
    #fig.savefig(figfile)
    
    plt.close()

figdir

1958 : 1959 : 1960 : 1961 : 1962 : 1963 : 1964 : 1965 : 1966 : 1967 : 1968 : 1969 : 1970 : 1971 : 1972 : 1973 : 1974 : 1975 : 1976 : 1977 : 1978 : 1979 : 1980 : 1981 : 1982 : 1983 : 1984 : 1985 : 1986 : 1987 : 1988 : 1989 : 1990 : 1991 : 1992 : 1993 : 1994 : 1995 : 1996 : 1997 : 1998 : 1999 : 2000 : 2001 : 2002 : 2003 : 2004 : 2005 : 2006 : 2007 : 2008 : 2009 : 2010 : 2011 : 2012 : 2013 : 2014 : 2015 : 2016 : 2017 : 2018 : 2019 : 2020 : 2021 : 

'/home/guillaumechagnaud/TRAVAIL/POSTDOC/IGE-Secheresse/work/output/figures/Fr/SAFRAN/1958-2021'